<a href="https://colab.research.google.com/github/samanthaorui/files-geosampa/blob/main/Mar%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install geopandas fiona pyproj shapely pandas



In [39]:
bboxes = {
    "centro_norte": "-4870000,-2745000,-4800000,-2695000",
    "zona_sul":     "-4845000,-2760000,-4795000,-2720000",
    "oeste_1":      "-4910000,-2760000,-4860000,-2710000",
    "oeste_2":      "-4860000,-2760000,-4810000,-2710000",
}


In [40]:
import geopandas as gpd
import time

def baixar_bbox(bbox, count=2000, tentativas=3):
    partes = []
    start = 0

    while True:
        url = (
            "https://tubvsig-so2sat-vm1.srv.mwn.de/geoserver/ows?"
            "service=WFS&version=2.0.0&request=GetFeature&"
            "typeNames=global3D:lod1_global&"
            "srsName=EPSG:3857&"
            f"bbox={bbox},EPSG:3857&"
            f"count={count}&startIndex={start}"
        )

        for _ in range(tentativas):
            try:
                gdf = gpd.read_file(url)
                break
            except Exception:
                time.sleep(2)
        else:
            return partes

        if gdf.empty:
            break

        if gdf.crs is None:
            gdf = gdf.set_crs("EPSG:3857", allow_override=True)

        partes.append(gdf)

        if len(gdf) < count:
            break

        start += count
        time.sleep(1)

    return partes


In [41]:
import pandas as pd

dfs = []

for name, bbox in bboxes.items():
    print(f"Baixando {name}...")
    partes = baixar_bbox(bbox)
    print(f"  → {sum(len(p) for p in partes)} edifícios")
    dfs.extend(partes)


Baixando centro_norte...
  → 0 edifícios
Baixando zona_sul...
  → 0 edifícios
Baixando oeste_1...
  → 0 edifícios
Baixando oeste_2...
  → 0 edifícios


In [42]:
import geopandas as gpd

if len(dfs) == 0:
    print("❌ Nenhum edifício foi baixado")
else:
    gdf_rio = gpd.GeoDataFrame(
        pd.concat(dfs, ignore_index=True),
        crs="EPSG:3857"
    )

    print("TOTAL FINAL:", len(gdf_rio))


❌ Nenhum edifício foi baixado


In [44]:
gdf_rio.to_file(
    "rio_de_janeiro_buildings.geojson",
    driver="GeoJSON"
)


NameError: name 'gdf_rio' is not defined

In [45]:
print("Quantidade de blocos baixados:", len(dfs))


Quantidade de blocos baixados: 0


In [46]:
import geopandas as gpd
import pandas as pd

if len(dfs) == 0:
    raise RuntimeError(
        "Nenhum edifício foi baixado. "
        "O GBA não tem cobertura significativa nessa área."
    )

gdf_rio = gpd.GeoDataFrame(
    pd.concat(dfs, ignore_index=True),
    crs="EPSG:3857"
)

print("TOTAL FINAL:", len(gdf_rio))

gdf_rio.to_file(
    "rio_de_janeiro_buildings.geojson",
    driver="GeoJSON"
)


RuntimeError: Nenhum edifício foi baixado. O GBA não tem cobertura significativa nessa área.